#Read in CPLEX Solution files
And make data to style maps in the multi-objective visualization

In [2]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os

In [3]:
frontiers = ["NONE", "Ensemble_rcp45", "Ensemble_rcp85"]
objNames = ["FireHazardIncrease","MinOwlHabitat","MaxSediment"]
# allSolutions is the data file fed to the visualization to create the scatterplots, parallel coords, etc.
allSolutionsRow = {'Frontier': [],
                'SolutionIndex': [],
                'Increase in fire hazard': [],
                'Min NSO habitat': [],
                'Max Sediment delivery (t)': []}
allSolutions = pd.DataFrame(allSolutionsRow)

In [99]:
# make empty datframe for stand-level storage of initial-final values for the objectives
initFinalStandVals = {'Stand': [],
                      'TimePeriod': [],
                      'Objective': [],
                      'Value': []}
initFinalStandVals = pd.DataFrame(initFinalStandVals)
initfinalfire = pd.read_csv("initialFinalStandFireData.csv")
initfinalowl = pd.read_csv("initialFinalStandOwlData.csv")
# list of objectives for which we will show initial and final statistics for
initFinalObjs = ["FireHazardIncrease","MinOwlHabitat"]

In [15]:
# make empty dataframe for storage of data for map coloring
valsForPaintingStandsInMapsEmpty = {'Frontier': [],
                               'SolutionIndex': [],
                               'MapColumn': [],
                               'Stand': [],
                               'Objective': [],
                               'Value': []}
valsForPaintingStandsInMaps = pd.DataFrame(valsForPaintingStandsInMapsEmpty)
#solutionsAlreadyInMapFile = [str(int(i)) for i in pd.read_csv("mapMakingData_1_upTo27.csv")["SolutionIndex"].unique().tolist()]
#solutionsAlreadyInMapFile

['0',
 '1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28']

In [6]:
def parseSolutionFiles():
    global allSolutions
    global valsForPaintingStandsInMaps
    # each solution will have a line in the allSolutions file
    newAllSolutionsRow = allSolutionsRow
    newAllSolutionsRow["Frontier"] = frontier
    # It will also have dataframe of length mapcols(4)*stands(303)*initFinalObjs(2)
    standMapVals = valsForPaintingStandsInMapsEmpty
    standMapVals["Frontier"] = frontier
    # get the solution index on the frontier, add it to our new row in the allSolutions file
    solIndex = solFile[solFile.rfind("_")+1:solFile.rfind(".")]
    standMapVals["SolutionIndex"] = solIndex
    newAllSolutionsRow["SolutionIndex"] = solIndex
    # rename the file accordingly
    newName = frontier + "_" + solIndex + ".sol"
    os.rename(solDir + solFile, solDir + newName)
    # parse the solution file
    print ("----------")
    print ("reading file " + newName + "...")
    solTree = ET.parse(solDir + newName)
    print (newName + " read successfully")
    cpxSol = solTree.getroot()
    # get important variable values: x_i_r's, and objectives
    numvarsSet = 0 # break loop when it hits 306
    numobjvarsSet = 0
    for variable in cpxSol.iter("variable"):
        varname = variable.get("name")
        firstCharOfName = varname[0].lower()
        if varname in objNames:
            print ("handling objective variable " + varname)
            varval = float(variable.get("value"))
            # store values in the new entry in the allSolutions row
            if varname[0:3] == "Fir":
                newAllSolutionsRow["Increase in fire hazard"] = varval
                numvarsSet += 1
                numobjvarsSet += 1
            elif varname[0:3] == "Min":
                newAllSolutionsRow["Min NSO habitat"] = varval
                numvarsSet += 1
                numobjvarsSet += 1
            else:
                newAllSolutionsRow["Max Sediment delivery (t)"] = varval
                numvarsSet += 1
                numobjvarsSet += 1
        # if all objective values have been recorded, determine if the solution is dominated
        fireobjval = newAllSolutionsRow["Increase in fire hazard"]
        nsoobjval = newAllSolutionsRow["Min NSO habitat"]
        sedobjval = newAllSolutionsRow["Max Sediment delivery (t)"]
        if numobjvarsSet == 3:
            # check dominated
            for idx,row in allSolutions.iterrows():
                checkvalfire = row["Increase in fire hazard"]
                checkvalowl = row["Min NSO habitat"]
                checkvalsed = row["Max Sediment delivery (t)"]
                if checkvalfire <= fireobjval and checkvalowl >= nsoobjval and checkvalsed <= sedobjval:
                    return
        if firstCharOfName == "x":
            varval = int(variable.get("value"))
            if varval > 0: # gives the stand's prescription
                print ("handling nonzero stand assignment variable " + varname)
                stand = int(varname[varname.index("_")+1:varname.rfind("_")])
                trtSched = int(varname[varname.rfind("_")+1])
                standMapVals["Stand"] = stand
                for mapcol in range(4):
                    standMapVals["MapColumn"] = mapcol
                    for obj in initFinalObjs:
                        standMapVals["Objective"] = obj
                        if mapcol == 0: # then we need to describe the initial state
                            if obj[0] == "F": # if doing the fire objective
                                standMapVals["Value"] = initfinalfire.loc[initfinalfire["stand"] == stand].loc[
                                    initfinalfire["climateScenario"] == frontier]["initial"].tolist()[0]
                            else: #doing the owl objective
                                if (initfinalowl.loc[initfinalowl["stand"] == stand].loc[
                                    initfinalowl["climateScenario"] == frontier]["initial"].tolist()[0]): # if true, assign value 1
                                    standMapVals["Value"] = 1
                                else:  # otherwise, assign value 0
                                    standMapVals["Value"] = 0
                        elif mapcol == 3: # then we need to describe the final state
                            if obj[0] == "F": # if doing the fire objective
                                standMapVals["Value"] = initfinalfire.loc[initfinalfire["stand"] == stand].loc[
                                    initfinalfire["climateScenario"] == frontier].ix[:,trtSched+3].tolist()[0]
                            else: #doing the owl objective
                                if (initfinalowl.loc[initfinalowl["stand"] == stand].loc[
                                    initfinalowl["climateScenario"] == frontier].ix[:,trtSched+3].tolist()[0]): # if true, assign value 1
                                    standMapVals["Value"] = 1
                                else:  # otherwise, assign value 0
                                    standMapVals["Value"] = 0
                        elif mapcol == 1: # looking at first time period
                            if trtSched == 1 or trtSched == 3: # we will treat in the first
                                standMapVals["Value"] = 1
                            else: #no trtment in the first period
                                standMapVals["Value"] = 0
                        else: # looking at second time period
                            if trtSched == 2 or trtSched == 3: # we will treat in the second
                                standMapVals["Value"] = 1
                            else:
                                standMapVals["Value"] = 0
                        valsForPaintingStandsInMaps = valsForPaintingStandsInMaps.append(standMapVals, ignore_index=True)
                        valsForPaintingStandsInMaps.to_csv("mapMakingData.csv", index=None)
                numvarsSet += 1
        if numvarsSet >= 306:
            print "All stand and objective "
            break
    # append new solution row to the allSolutions df
    allSolutions = allSolutions.append(newAllSolutionsRow, ignore_index=True)
    allSolutions.to_csv("climateChange_AllSolutions_primary.csv", index=None)
    return

In [7]:
def parseSolutionFilesButOnlyGetObjectiveData():
    global allSolutions
    # each solution will have a line in the allSolutions file
    newAllSolutionsRow = allSolutionsRow
    newAllSolutionsRow["Frontier"] = frontier
    # get the solution index on the frontier, add it to our new row in the allSolutions file
    solIndex = solFile[solFile.rfind("_")+1:solFile.rfind(".")]
    newAllSolutionsRow["SolutionIndex"] = solIndex
    # rename the file accordingly
    newName = frontier + "_" + solIndex + ".sol"
    os.rename(solDir + solFile, solDir + newName)
    # parse the solution file
    print ("----------")
    print ("reading file " + newName + "...")
    solTree = ET.parse(solDir + newName)
    print (newName + " read successfully")
    cpxSol = solTree.getroot()
    # get important variable values: x_i_r's, and objectives
    numvarsSet = 0 # break loop when it hits 306
    numobjvarsSet = 0
    for variable in cpxSol.iter("variable"):
        varname = variable.get("name")
        firstCharOfName = varname[0].lower()
        if varname in objNames:
            print ("handling objective variable " + varname)
            varval = float(variable.get("value"))
            # store values in the new entry in the allSolutions row
            if varname[0:3] == "Fir":
                newAllSolutionsRow["Increase in fire hazard"] = varval
                numvarsSet += 1
                numobjvarsSet += 1
            elif varname[0:3] == "Min":
                newAllSolutionsRow["Min NSO habitat"] = varval
                numvarsSet += 1
                numobjvarsSet += 1
            else:
                newAllSolutionsRow["Max Sediment delivery (t)"] = varval
                numvarsSet += 1
                numobjvarsSet += 1
        # if all objective values have been recorded, determine if the solution is dominated
        fireobjval = newAllSolutionsRow["Increase in fire hazard"]
        nsoobjval = newAllSolutionsRow["Min NSO habitat"]
        sedobjval = newAllSolutionsRow["Max Sediment delivery (t)"]
        if numobjvarsSet == 3:
            # check dominated
            for idx,row in allSolutions.iterrows():
                checkvalfire = row["Increase in fire hazard"]
                checkvalowl = row["Min NSO habitat"]
                checkvalsed = row["Max Sediment delivery (t)"]
                if checkvalfire <= fireobjval and checkvalowl >= nsoobjval and checkvalsed <= sedobjval:
                    return
            # then since it's not dominated, print the new line and return
            # append new solution row to the allSolutions df
            allSolutions = allSolutions.append(newAllSolutionsRow, ignore_index=True)
            allSolutions = allSolutions[["Frontier","SolutionIndex","Increase in fire hazard","Max Sediment delivery (t)","Min NSO habitat"]]
            allSolutions.to_csv("climateChange_AllSolutions_primary.csv", index=None)
            return

In [33]:
def parseSolutionFilesButJustMapFileStuff():
    global valsForPaintingStandsInMaps
    # It will have dataframe of length mapcols(4)*stands(303)*initFinalObjs(2)
    standMapVals = valsForPaintingStandsInMapsEmpty
    standMapVals["Frontier"] = frontier
    # get the solution index on the frontier
    solIndex = solFile[solFile.rfind("_")+1:solFile.rfind(".")]
    if solIndex != '28':
        print "not 28. terminating"
        return
    print solIndex
    standMapVals["SolutionIndex"] = solIndex
    # rename the file accordingly
    newName = frontier + "_" + solIndex + ".sol"
    os.rename(solDir + solFile, solDir + newName)
    # parse the solution file
    print ("----------")
    print ("reading file " + newName + "...")
    solTree = ET.parse(solDir + newName)
    print (newName + " read successfully")
    cpxSol = solTree.getroot()
    # get important variable values: x_i_r's, and objectives
    numvarsSet = 0 # break loop when it hits 303
    numobjvarsSet = 0
    for variable in cpxSol.iter("variable"):
        varname = variable.get("name")
        firstCharOfName = varname[0].lower()
        if firstCharOfName == "x":
            varval = int(round(float(variable.get("value"))))
            if varval > 0: # gives the stand's prescription
                print ("handling nonzero stand assignment variable " + varname)
                stand = int(varname[varname.index("_")+1:varname.rfind("_")])
                trtSched = int(varname[varname.rfind("_")+1])
                standMapVals["Stand"] = stand
                for mapcol in range(4):
                    standMapVals["MapColumn"] = mapcol
                    for obj in initFinalObjs:
                        standMapVals["Objective"] = obj
                        if mapcol == 0: # then we need to describe the initial state
                            if obj[0] == "F": # if doing the fire objective
                                standMapVals["Value"] = initfinalfire.loc[initfinalfire["stand"] == stand].loc[
                                    initfinalfire["climateScenario"] == frontier]["initial"].tolist()[0]
                            else: #doing the owl objective
                                if (initfinalowl.loc[initfinalowl["stand"] == stand].loc[
                                    initfinalowl["climateScenario"] == frontier]["initial"].tolist()[0]): # if true, assign value 1
                                    standMapVals["Value"] = 1
                                else:  # otherwise, assign value 0
                                    standMapVals["Value"] = 0
                        elif mapcol == 3: # then we need to describe the final state
                            if obj[0] == "F": # if doing the fire objective
                                standMapVals["Value"] = initfinalfire.loc[initfinalfire["stand"] == stand].loc[
                                    initfinalfire["climateScenario"] == frontier].ix[:,trtSched+3].tolist()[0]
                            else: #doing the owl objective
                                if (initfinalowl.loc[initfinalowl["stand"] == stand].loc[
                                    initfinalowl["climateScenario"] == frontier].ix[:,trtSched+3].tolist()[0]): # if true, assign value 1
                                    standMapVals["Value"] = 1
                                else:  # otherwise, assign value 0
                                    standMapVals["Value"] = 0
                        elif mapcol == 1: # looking at first time period
                            if trtSched == 1 or trtSched == 3: # we will treat in the first
                                standMapVals["Value"] = 1
                            else: #no trtment in the first period
                                standMapVals["Value"] = 0
                        else: # looking at second time period
                            if trtSched == 2 or trtSched == 3: # we will treat in the second
                                standMapVals["Value"] = 1
                            else:
                                standMapVals["Value"] = 0
                        valsForPaintingStandsInMaps = valsForPaintingStandsInMaps.append(standMapVals, ignore_index=True)
                numvarsSet += 1
        if numvarsSet >= 303:
            print "All stands in this solution accounted for. Reprinting CSV and moving on."
            valsForPaintingStandsInMaps.to_csv("mapMakingData_only28.csv", index=None)
            break
    return

In [34]:
for frontier in frontiers:
    print frontier
    if frontier.startswith("E"): # temp break while trialing the below
        break
    # get list of solution files from this folder
    solFiles = []
    solDir = "solutionFiles/" + frontier + "/"
    for (dirpath, dirnames, filenames) in os.walk(solDir):
        solFiles.extend(filenames)
    # for each solution...
    for solFile in solFiles:
        parseSolutionFilesButJustMapFileStuff()

NONE
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
not 28. terminating
28
----------
reading file NONE_28.sol...
NONE_28.sol read successfully
handling nonzero stand assignment variable x_0_1
handling nonzero stand assignment 

In [107]:
#objNames.index("MinOwlHabitat")
#allSolutionsRow["Increase in fire hazard"] = 12
#initfinalfire.loc[initfinalfire["stand"] == 1].loc[initfinalfire["climateScenario"] == "NONE"].ix[:,trtSched+3].tolist()[0]
#domsoltester = pd.read_csv("../../visualization/data/frontiers.csv")
#domsoltester = domsoltester.append(domsoltester.ix[0], ignore_index=True)
#o1val = domsoltester.ix[0,2]
#o2val = domsoltester.ix[0,3]
#o3val = domsoltester.ix[0,4]
#ovals = [o1val,o2val,o3val]
#for idx,row in domsoltester.iterrows():
#    checkval1 = row[2]
#    checkval2 = row[3]
#    checkval3 = row[4]
#    if checkval1 > 
#    print checkval1
#valsForPaintingStandsInMaps
allmapdata = pd.read_csv("mapMakingData_all.csv")
allmapdata["MapColumn"] = allmapdata["MapColumn"].astype(int)
allmapdata["SolutionIndex"] = allmapdata["SolutionIndex"].astype(int)
allmapdata["Stand"] = allmapdata["Stand"].astype(int)
allmapdata["Frontier"] = "NONE"
# There are some issues in the values column. Fixing them now.
# Determined that the NaNs are from stands that were not simulated (not forested)
# Their NaN values can be replaced with 0s
allmapdata["Value"] = allmapdata["Value"].fillna(0)
allmapdata.to_csv("mapMakingData_noneOnly_final.csv", index=None)

In [100]:
#int(round(float('1.0000000000004092')))
#tdf = allmapdata
#tdf.ix[index[0],"Frontier"] = "test!"
#min(tdf["Stand"].unique().tolist())
initfinalfire.loc[initfinalfire["stand"] == 249]

,stand,climateScenario,initial,none,first,second,both
747,249,NONE,0,0,0,0,0
748,249,Ensemble_rcp45,0,0,0,0,0
749,249,Ensemble_rcp85,0,0,0,0,0


In [40]:
# output datafile should ideally be something like:
# frontier | solnNum | Map Col (intial, first period, second period, end) | siteNum | value (to determine coloring)
# if map column is first or last, then color scheme is different than if it's just during the middle
# first/last show some quantitative variable on each stand
# middle show binary (yes treated or no treated)
# should be able to sniff out which color scheme to do based on the row's value for map col